In [27]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [28]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [34]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():
        lmk_x, lmk_y, lmk_z = 0, 0, 0
        wlmk_x, wlmk_y, wlmk_z = 0, 0, 0

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                lmk_x, lmk_y, lmk_z = [[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark][0]

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        if results.multi_hand_world_landmarks:
            for hand_landmarks in results.multi_hand_world_landmarks:
                wlmk_x, wlmk_y, wlmk_z = [[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark][0]


        frame = cv2.copyMakeBorder(frame, 0, 80, 0, 0, cv2.BORDER_CONSTANT)

        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        frame = cv2.putText(frame, f"L: {lmk_x:.4f}, {lmk_y:.4f}, {lmk_z:.4f}", (10,frame.shape[1]-90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        frame = cv2.putText(frame, f"WL: {wlmk_x:.4f}, {wlmk_y:.4f}, {wlmk_z:.4f}", (10,frame.shape[1]-130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [30]:
cap.release()
cv2.destroyAllWindows()